In [1]:
from pyspark import SparkContext
from scipy import sparse as sm
from sklearn.preprocessing import normalize
import numpy as np
import csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from scipy.stats import spearmanr
from scipy.stats import pearsonr as pears
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
import time
from sparsesvd import sparsesvd
import math as mt
sc = SparkContext.getOrCreate()

In [2]:
train_rdd = sc.textFile("data/train.csv")
icm_rdd = sc.textFile("data/icm_fede.csv")
test_rdd= sc.textFile("data/target_users.csv")

train_header = train_rdd.first()
icm_header = icm_rdd.first()
test_header= test_rdd.first()

train_clean_data = train_rdd.filter(lambda x: x != train_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1]), float(x[2])))
icm_clean_data = icm_rdd.filter(lambda x: x != icm_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1])))
test_clean_data= test_rdd.filter(lambda x: x != test_header).map(lambda line: line.split(','))

test_users=test_clean_data.map( lambda x: int(x[0])).collect()


grouped_rates = train_clean_data.filter(lambda x: x[0] in test_users).map(lambda x: (x[0],x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()
grouped_rates_dic = dict(grouped_rates)


item_ratings = train_clean_data.map(lambda x: (x[0], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))
user_ratings_mean = item_ratings.mapValues(lambda x: (x[0] / (x[1]))).collect()
user_ratings_mean_dic=dict(user_ratings_mean)


item_ratings_forTop = train_clean_data.map(lambda x: (x[1], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))#.sortBy(lambda x: x[1][1], ascending=False)
shrinkage_factor = 5
item_ratings_mean = item_ratings_forTop.mapValues(lambda x: (x[0] / (x[1] + shrinkage_factor))).sortBy(lambda x: x[1], ascending = False).map(lambda x: x[0]).collect()


users = train_clean_data.map(lambda x: x[0]).collect()
items = train_clean_data.map(lambda x: x[1]).collect()
ratings = train_clean_data.map(lambda x: x[2]).collect()
#ratings_unbiased = train_clean_data.map(lambda x: x[2]-user_ratings_mean_dic[x[0]]).collect()

items_for_features= icm_clean_data.map(lambda x:x[0]).collect()
features = icm_clean_data.map(lambda x:x[1]).collect()
items_for_features.append(37142)
features.append(0)


unos=[1]*len(items_for_features)

UxI= sm.csr_matrix((ratings, (users, items)))
#UxI_unbiased= sm.csr_matrix((ratings_unbiased, (users, items)))
IxF= sm.csr_matrix((unos, (items_for_features, features)))

In [3]:
n_users,n_items=UxI.shape
n_features=IxF.shape[1]

In [4]:
'''SVD'''
K = 870
%time U, S, Vt = sparsesvd(UxI.tocsc(), K)


CPU times: user 1min 52s, sys: 156 ms, total: 1min 52s
Wall time: 1min 51s


In [5]:
S=np.sqrt(S.data)
S=np.diag(S)

In [6]:
#calc predictions
%time UxI_pred_SVD=sm.csr_matrix(U.T.dot(S).dot(Vt))

CPU times: user 48.7 s, sys: 5.93 s, total: 54.6 s
Wall time: 28 s


In [7]:
'''content based'''
IDF=[0]*n_features
for i in tqdm(range(n_features)):
    IDF[i]=np.log10(n_items/len(IxF.getcol(i).nonzero()[1]))
%time IxF=normalize(IxF,axis=1)
%time IxF_idf=IxF.multiply(IDF)
%time UxF=UxI.dot(IxF_idf)
%time UxI_pred_CB=UxF.dot(IxF.T)

Widget Javascript not detected.  It may not be installed or enabled properly.



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.33 ms
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.43 ms
CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 60.8 ms


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


CPU times: user 8.22 s, sys: 1.05 s, total: 9.27 s
Wall time: 9.27 s


In [8]:
del IxF

In [9]:
'''collaborative filtering item based via content'''
#calc similarities
%time IxI_sim=IxF_idf.dot(IxF_idf.T)
IxI_sim.setdiag(0)

CPU times: user 6.73 s, sys: 1.49 s, total: 8.22 s
Wall time: 8.22 s


/home/ubuntu/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [10]:
#take knn items
IxI_sim_knn=sm.lil_matrix((n_items,n_items))
k=200
for i in tqdm(range(n_items)):    
    top_k_idx =IxI_sim.getrow(i).toarray()[0].argpartition(-k)[-k:]
    IxI_sim_knn[i,top_k_idx]=IxI_sim[i,top_k_idx]

Widget Javascript not detected.  It may not be installed or enabled properly.


In [11]:
del IxI_sim 

In [12]:
#calc predictions
%time UxI_pred_CI=UxI.dot(IxI_sim_knn.T)

CPU times: user 3.32 s, sys: 372 ms, total: 3.69 s
Wall time: 3.68 s


In [13]:
del IxI_sim_knn

In [14]:
'''collaborative filtering user based via content'''
#calc similarities
%time UxU_sim=UxF.dot(UxF.T) #numerators of cosine

CPU times: user 5.87 s, sys: 512 ms, total: 6.38 s
Wall time: 6.38 s


In [15]:
%time cos=sm.csr_matrix(cosine_similarity(UxF)) #cosine
cos.data=1/cos.data
denominators=UxU_sim.multiply(cos) #get denominators of cosine
del cos

CPU times: user 14.9 s, sys: 3.17 s, total: 18.1 s
Wall time: 18.1 s


In [16]:
denominators.data+=1 #add shrinkage to cosine denominator
denominators.data=1/denominators.data
%time UxU_sim=UxU_sim.multiply(denominators) #calc shrinked cosine
del denominators

CPU times: user 2.44 s, sys: 552 ms, total: 2.99 s
Wall time: 2.99 s


In [17]:
UxU_sim.setdiag(0)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [18]:
#take knn users
UxU_sim_knn=sm.lil_matrix((n_users,n_users))
k=50
for i in tqdm(range(n_users)):    
    top_k_idx =UxU_sim.getrow(i).toarray()[0].argpartition(-k)[-k:]
    UxU_sim_knn[i,top_k_idx]=UxU_sim[i,top_k_idx]  

Widget Javascript not detected.  It may not be installed or enabled properly.


In [19]:
del UxU_sim

In [20]:
#calc_predictions
%time UxI_pred_CU=UxU_sim_knn.dot(UxI) #k=50->382

CPU times: user 1.26 s, sys: 100 ms, total: 1.36 s
Wall time: 1.36 s


In [21]:
del UxU_sim_knn
del UxI
del UxF
del IxF_idf

In [ ]:
'''aaaaaaaaaalllllttttt rimuovere già votati con lil altrimenti ci mette 10 minuti... ma convertendo a lil il kernel muore'''
''''''

#remove already voted
for user in tqdm(test_users):
    UxI_pred_CB[user,grouped_rates_dic[user]]=0
    UxI_pred_CI[user,grouped_rates_dic[user]]=0
    UxI_pred_CU[user,grouped_rates_dic[user]]=0
    UxI_pred_SVD[user,grouped_rates_dic[user]]=0

#rescale algorithm 1
UxI_pred_CB_temp=sm.lil_matrix((n_users,n_items))
for user in tqdm(test_users):       
    OldMax=max(UxI_pred_CB.getrow(user).toarray()[0])
    if(OldMax!=0):
        UxI_pred_CB_temp[user,:]=(10*(UxI_pred_CB[user,:]/OldMax)) 
%time UxI_pred_CB=UxI_pred_CB_temp.tocsr()
del UxI_pred_CB_temp

#rescale algorithm 2
UxI_pred_CI_temp=sm.lil_matrix((n_users,n_items))
for user in tqdm(test_users):       
    OldMax=max(UxI_pred_CI.getrow(user).toarray()[0])
    if(OldMax!=0):
        UxI_pred_CI_temp[user,:]=(10*(UxI_pred_CI[user,:]/OldMax)) 
%time UxI_pred_CI=UxI_pred_CI_temp.tocsr()
del UxI_pred_CI_temp

#rescale algorithm 3
UxI_pred_CU_temp=sm.lil_matrix((n_users,n_items))
for user in tqdm(test_users):       
    OldMax=max(UxI_pred_CU.getrow(user).toarray()[0])
    if(OldMax!=0):
        UxI_pred_CU_temp[user,:]=(10*(UxI_pred_CU[user,:]/OldMax)) 
%time UxI_pred_CU=UxI_pred_CU_temp.tocsr()
del UxI_pred_CU_temp

#rescale algorithm 4
UxI_pred_SVD_temp=sm.lil_matrix((n_users,n_items))
for user in tqdm(test_users):       
    OldMax=max(UxI_pred_SVD.getrow(user).toarray()[0])
    if(OldMax!=0):
        UxI_pred_SVD_temp[user,:]=(10*(UxI_pred_SVD[user,:]/OldMax)) 
%time UxI_pred_SVD=UxI_pred_SVD_temp.tocsr()
del UxI_pred_SVD_temp

In [22]:
#rescale algoritms
%time UxI_pred_CB/=UxI_pred_CB.max()
%time UxI_pred_CI/=UxI_pred_CI.max()
%time UxI_pred_CU/=UxI_pred_CU.max()
%time UxI_pred_SVD/=UxI_pred_SVD.max()

CPU times: user 30 s, sys: 0 ns, total: 30 s
Wall time: 30 s
CPU times: user 968 ms, sys: 0 ns, total: 968 ms
Wall time: 966 ms
CPU times: user 2.68 s, sys: 0 ns, total: 2.68 s
Wall time: 2.67 s
CPU times: user 1.34 s, sys: 0 ns, total: 1.34 s
Wall time: 1.34 s


Predictions of all algorithms computed, now have fun with hybrids!

In [28]:
%time UxI_pred=UxI_pred_CB*0.80+UxI_pred_SVD*0.20


CPU times: user 5.71 s, sys: 3.55 s, total: 9.26 s
Wall time: 9.26 s


In [29]:
%time UxI_pred=UxI_pred*0.80+UxI_pred_CI*0.20

CPU times: user 3.24 s, sys: 2.78 s, total: 6.02 s
Wall time: 6.02 s


In [30]:
f = open('submission_wSUM_CB-SVD_CI_75-25_80_20.csv', 'wt')
#f = open('submission_CBBBB.csv', 'wt')
writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))

for user in tqdm(test_users):
    top=[]

    user_predictions=user_predictions=(-UxI_pred.getrow(user).toarray()[0]).argsort()
    i=0
    while len(top)<=5:
        prediction=user_predictions[i]
        
        if UxI_pred[user,prediction]==0.0:
            j=0
            while len(top)<=5:
                prediction = item_ratings_mean[j]
                while prediction in grouped_rates_dic[user] or prediction in top:
                    j += 1
                    prediction = item_ratings_mean[j]
                top.append(prediction)
                j += 1
        
        
        
        if prediction in grouped_rates_dic[user]:
            i+=1
            continue
        
        i+=1
        top.append(prediction)
        
   
    writer.writerow((user, '{0} {1} {2} {3} {4}'.format(top[0], top[1], top[2], top[3], top[4])))

f.close()

In [ ]:
def bordaLikeAggr(rank1,rank2):
    result=[0]*n_items
    rg=150
    for i in range(rg):
        item1=rank1.argmax()
        item2=rank2.argmax()        
        #item3=rank3.argmax()
        #item3=rank3.argmax()

        if rank1[item1]>0.0:
            result[item1]+=(0.75/(i+1))
        rank1[item1]=-9

        if rank2[item2]>0.0:
            result[item2]+=(0.25/(i+1))
        rank2[item2]=-9
        '''
        if rank3[item3]>0.0:
            result[item3]+=(0.25/(i+1))
        rank3[item3]=-9
        
        if rank4[item4]>0.0:
            result[item4]+=(0.25/(i+1))
        rank4[item4]=-9'''

    return sm.csr_matrix(result)

In [ ]:
f = open('submission_borda_CI-SVD_75-25.csv', 'wt')
writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))
for user in tqdm(test_users):
    top=[0,0,0,0,0]

    user_predictions=bordaLikeAggr(UxI_pred_CI.getrow(user).toarray()[0],UxI_pred_SVD.getrow(user).toarray()[0])
    iterator = 0
    for i in range(5):
        prediction = user_predictions.argmax()
        while prediction in grouped_rates_dic[user] and prediction != 0:
            user_predictions[0,prediction]=-9
            prediction=user_predictions.argmax()
        if prediction == 0:
            prediction = item_ratings_mean[iterator]
            while prediction in grouped_rates_dic[user] or prediction in top:
                iterator += 1
                prediction = item_ratings_mean[iterator]
            iterator += 1
        else:
            user_predictions[0,prediction]=-9
        top[i]=prediction    
    writer.writerow((user, '{0} {1} {2} {3} {4}'.format(top[0], top[1], top[2], top[3], top[4])))

f.close()

In [ ]:
UxI_pred_CB[4,:].max()